# **Recomendador**

Utilizando el pre procesamiento de los dataframe, el clasificador de emociones, en este colab se desarrolla el recomendador segun los criterios establecidos.

In [ ]:
!pip install transformers sentence_transformers
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=8e372eb88776e528e3de345f8462e5bd853fb3325094e0187a1c4d10539bfadb
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

In [ ]:
!pip install gliner
from gliner import GLiNER

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from googletrans import Translator
import joblib

In [ ]:
# Funcion para traducir frases en castellano a ingles

translator = Translator()
def translate(text):
  translation = translator.translate(text, src='es', dest='en')
  return translation.text

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(250105, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-11): 12 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): StableDropout()
                    (dropout): StableDropout()
                  )
                  (output): Debert

In [ ]:
# Función para extraer entidades nombradas relevantes del texto de consulta

# labels para hacer la busqueda
labels = ["genre", "author", "title", "director", "actor"]

def extract_entities(text):
    entities = {
        "genre": [],
        "author": [],
        "title": [],
        "director": [],
        "actor": []
    }
    entities_ = model.predict_entities(text, labels, threshold=0.4)

    for ent in entities_:
      entities[ent['label']].append(ent['text'].lower())
    return entities

In [ ]:
extract_entities("I want to read Science Fiction books from Stephen King, as The Green Mill. Or maybe watch movies like Titanic, from James Cameron with Leonardo Di Caprio")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'genre': ['science fiction'],
 'author': ['stephen king'],
 'title': ['the green mill', 'titanic'],
 'director': ['james cameron'],
 'actor': ['leonardo di caprio']}

In [ ]:
extract_entities("I would like to play Pandemic Legacy")

{'genre': [],
 'author': [],
 'title': ['pandemic legacy'],
 'director': [],
 'actor': []}

In [ ]:
modelo = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Juegos de mesa BGG**

In [ ]:
bgg_df = pd.read_csv('/content/drive/MyDrive/NLP/bgg_finish.csv')
bgg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              1000 non-null   int64  
 1   game_name         1000 non-null   object 
 2   game_href         1000 non-null   object 
 3   geek_rating       1000 non-null   float64
 4   avg_rating        1000 non-null   float64
 5   num_voters        1000 non-null   float64
 6   description       1000 non-null   object 
 7   yearpublished     1000 non-null   int64  
 8   minplayers        1000 non-null   int64  
 9   maxplayers        1000 non-null   int64  
 10  minplaytime       1000 non-null   int64  
 11  maxplaytime       1000 non-null   int64  
 12  minage            1000 non-null   int64  
 13  avgweight         1000 non-null   float64
 14  best_num_players  1000 non-null   object 
 15  designers         1000 non-null   object 
 16  mechanics         1000 non-null   object 
 

In [ ]:
# Funcion que recibe un embedding de la frase del usuario, el dataframe y los embeddings de dicho dataframe
# Retrorna las 5 mejores recomendaciones de acuerdo a la frase

embeddings_bgg = modelo.encode(bgg_df['Combined'].tolist(), convert_to_tensor=True)

def recommend_bgg(frase_usuario, df):
    frase_usuario = translate(frase_usuario)
    embedding_usuario = modelo.encode(frase_usuario, convert_to_tensor=True)

    filtered_df = df
    entities_usuario = extract_entities(frase_usuario)
    if entities_usuario['title']:
      filtered_df = filtered_df[filtered_df["game_name"].str.lower().str.contains('|'.join(entities_usuario["title"]), na=False)]


    if len(filtered_df.index) > 0:
      if len(filtered_df.index) <= 5:
        return filtered_df
      else:
        df = filtered_df

    similitudes = util.pytorch_cos_sim(embedding_usuario, embeddings_bgg)

    indices_similares = similitudes[0].argsort(descending=True)
    num_rows = df.shape[0]
    indices_validos = [i for i in indices_similares if i < num_rows]

    return df.iloc[indices_validos][:5]

In [ ]:
# Ejemplo de recomendaciones

frase_usuario = "Me gustaria jugar Pandemic Legacy o algo de aventura"

juegos_recomendados = recommend_bgg(frase_usuario, bgg_df)

juegos_recomendados['game_name']

**Peliculas IMDB**

In [ ]:
imdb_df = pd.read_csv('/content/drive/MyDrive/NLP/imdb_finish.csv')
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  1000 non-null   float64
 11  Metascore           1000 non-null   int64  
 12  emocion             1000 non-null   object 
 13  Combined            1000 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 109.5+ KB


In [ ]:
embeddings_imdb = modelo.encode(imdb_df['Combined'].tolist(), convert_to_tensor=True)

In [ ]:
# Funcion que recibe un embedding de la frase del usuario, el dataframe y los embeddings de dicho dataframe
# Retrorna las 5 mejores recomendaciones de acuerdo a la frase

def recommend_imdb(frase_usuario, df):
    frase_usuario = translate(frase_usuario)
    embedding_usuario = modelo.encode(frase_usuario, convert_to_tensor=True)

    filtered_df = df
    entities_usuario = extract_entities(frase_usuario)
    if entities_usuario['title']:
      filtered_df = filtered_df[filtered_df["Title"].str.lower().str.contains('|'.join(entities_usuario["title"]), na=False)]
      if len(filtered_df.index) == 0:
        filtered_df = df
    if entities_usuario['director']:
      filtered_df = filtered_df[filtered_df["Director"].str.lower().str.contains('|'.join(entities_usuario["director"]), na=False)]
      if len(filtered_df.index) == 0:
        filtered_df = df
    if entities_usuario['actor']:
      filtered_df = filtered_df[filtered_df["Actors"].str.lower().str.contains('|'.join(entities_usuario["actor"]), na=False)]
      if len(filtered_df.index) == 0:
        filtered_df = df
    if entities_usuario['genre']:
      filtered_df = filtered_df[filtered_df["Genre"].str.lower().str.contains('|'.join(entities_usuario["genre"]), na=False)]


    if len(filtered_df.index) > 0:
      if len(filtered_df.index) <= 5:
        return filtered_df
      else:
        df = filtered_df

    similitudes = util.pytorch_cos_sim(embedding_usuario, embeddings_imdb)

    indices_similares = similitudes[0].argsort(descending=True)

    num_rows = df.shape[0]
    indices_validos = [i for i in indices_similares if i < num_rows]
    return df.iloc[indices_validos][:5]

In [ ]:
# Ejemplo de recomendaciones

frase_usuario = "Me gustaria ver Titanic de James Cameron o alguna otra de amor y aventura"
juegos_recomendados = recommend_imdb(frase_usuario, imdb_df)

juegos_recomendados['Title']

,Title
87,Avatar


**Libros**

In [ ]:
books_df = pd.read_csv('/content/drive/MyDrive/NLP/books_finish.csv')
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      982 non-null    object
 1   summary     966 non-null    object
 2   subject     982 non-null    object
 3   titulo      960 non-null    object
 4   genero      982 non-null    object
 5   categories  982 non-null    object
 6   emocion     982 non-null    object
 7   Combined    966 non-null    object
dtypes: object(8)
memory usage: 61.5+ KB


In [ ]:
embeddings_books = modelo.encode(books_df['Combined'].tolist(), convert_to_tensor=True)

In [ ]:
# Funcion que recibe un embedding de la frase del usuario, el dataframe y los embeddings de dicho dataframe
# Retrorna las 5 mejores recomendaciones de acuerdo a la frase

def recommend_books(frase_usuario, df):
    frase_usuario = translate(frase_usuario)
    embedding_usuario = modelo.encode(frase_usuario, convert_to_tensor=True)

    filtered_df = df
    entities_usuario = extract_entities(frase_usuario)
    if entities_usuario['title']:
      filtered_df = filtered_df[filtered_df["titulo"].str.lower().str.contains('|'.join(entities_usuario["title"]), na=False)]
      if len(filtered_df.index) == 0:
        filtered_df = df
    if entities_usuario['author']:
      filtered_df = filtered_df[filtered_df["autor"].str.lower().str.contains('|'.join(entities_usuario["author"]), na=False)]
      if len(filtered_df.index) == 0:
        filtered_df = df
    if entities_usuario['genre']:
      filtered_df = filtered_df[filtered_df["genero"].str.lower().str.contains('|'.join(entities_usuario["genre"]), na=False)]

    if len(filtered_df.index) > 0:
      if len(filtered_df.index) <= 5:
        return filtered_df
      else:
        df = filtered_df

    similitudes = util.pytorch_cos_sim(embedding_usuario, embeddings_books)

    indices_similares = similitudes[0].argsort(descending=True)

    num_rows = df.shape[0]
    indices_validos = [i for i in indices_similares if i < num_rows]

    return df.iloc[indices_validos][:5]

In [ ]:
# Ejemplo de recomendaciones

frase_usuario = "Me gustaria algo entretenido, como leer un libro de misterio o thriller"
libros_recomendados = recommend_books(frase_usuario, books_df)

libros_recomendados['titulo']

,titulo
518,Tales of Terror and Mystery
491,The Purcell Papers — Volume 2
820,New Arabian Nights
764,The Lost City
171,The Haunted Bookshop


# **Recomendador final**

In [ ]:
model_class = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Carga del el modelo para clasificar la emocion desde el archivo .pkl

modelo_LR = joblib.load("/content/drive/MyDrive/NLP/modelo_LR.pkl")

In [ ]:
# Función para clasificar una frase y devolver la emoción
emociones = {
      '0': "tristeza",
      '1': "alegría",
      '3': "enojo"
    }

def clasificar_emocion(frase):
    frase_traducida = translate(frase)
    frase_lower = frase_traducida.lower()
    frase_vectorizada = model_class.encode([frase_lower])
    new_predictions = modelo_LR.predict(frase_vectorizada)[0]
    emocion = emociones.get(new_predictions, "Emoción desconocida")

    return emocion

In [ ]:
clasificar_emocion("Me siento con ganas de hacer nada")

'tristeza'

In [ ]:
## Funcion final, recomienda peliculas libros y juegos en base a 2 frases.
# la primer frase se utiliza para determinar la emocion y filtrar inicialmente los datasets
# la segunda se aplica NER y se compara por similitud para buscar la mejor recomendacion

def recomendar(frase_emocion, frase_usuario):
  emocion = clasificar_emocion(frase_emocion)
  print(f"La emocion encontrada es: {emocion}")

  bgg_df_filtered = bgg_df[bgg_df["emocion"] == emocion]
  juegos_recomendados = recommend_bgg(frase_usuario, bgg_df_filtered)
  print("Se recomiendan los siguientes juegos:")
  for index, row in juegos_recomendados.iterrows():
    print("*",  row['game_name'])

  imdb_df_filtered = imdb_df[imdb_df["emocion"] == emocion]
  pelis_recomendadas = recommend_imdb(frase_usuario, imdb_df_filtered)
  print("Se recomiendan las siguientes peliculas:")
  for index, row in pelis_recomendadas.iterrows():
    print("*", row['Title'])

  books_df_fitered = books_df[books_df["emocion"] == emocion]
  libros_recomendados = recommend_books(frase_usuario, books_df_fitered)
  print("Se recomiendan los siguientes libros")
  for index, row in libros_recomendados.iterrows():
    print("*", row['titulo'])

In [ ]:
frase_emocion = "hoy me siento con ganas de hacer algo divertido"
frase_usuario = "quisiera ver o leer algo de accion o aventura"
recomendar(frase_emocion, frase_usuario)

La emocion encontrada es: enojo
Se recomiendan los siguientes juegos:
* Undaunted: Normandy
* Tajemnicze Domostwo
* Hammer of the Scots
* Detective: A Modern Crime Board Game
* 1775: Rebellion
Se recomiendan las siguientes peliculas:
* Live by Night
* King Cobra
* The Ghost Writer
* Crawlspace
* Contratiempo
Se recomiendan los siguientes libros
* The Burial of the Guns
* The Haunted Man and the Ghost's Bargain
* At the Earth's Core
* Summer
* The Cricket on the Hearth: A Fairy Tale of Home


In [ ]:
frase_emocion = "hoy me siento motivado para hacer cosas"
frase_usuario = "Estoy interesado en algo como Titanic de James Cameron"
recomendar(frase_emocion, frase_usuario)

La emocion encontrada es: enojo
Se recomiendan los siguientes juegos:
* Unfathomable
* Forgotten Waters
* Nemo's War (Second Edition)
* Oceans
* Deep Sea Adventure
Se recomiendan las siguientes peliculas:
* Avatar
Se recomiendan los siguientes libros
* Sinking of the Titanic and Great Sea Disasters
